In [16]:
import pandas as pd
from sqlalchemy import engine, create_engine, sql, Column, Integer, String, Float, Date, func
from sqlalchemy.orm import declarative_base, sessionmaker
import yfinance as yf
import sys
from datetime import datetime, timedelta
from matplotlib import pyplot as plt

connection_url = engine.URL.create(
    "mssql",
    host="DATOR",
    database="Finance",
    query={
        "driver": "SQL Server",
        "TrustServerCertificate": "yes",
        #"authentication": "ActiveDirectoryIntegrated",
    },
)

engine = create_engine(connection_url)


In [2]:
Base = declarative_base()
class History(Base):
    __tablename__ = "History"
    Id = Column(Integer, primary_key=True)
    Open = Column(Float)
    Ticker = Column(String(50))
    Date = Column(Date)
    LastUpdated = Column(String)
    
    def __repr__(self):
        return "<History(Ticker='%s', Open='%s')>" % (self.Ticker, self.Open)

In [3]:
def full_load(ticker, truncate=False):
    
    if truncate:
        engine.execute(sql.text('''TRUNCATE TABLE History''').execution_options(autocommit=True))
    else:
        engine.execute(sql.text('''DELETE FROM History WHERE Ticker=:ticker''').params(ticker=ticker).execution_options(autocommit=True))
    
    t = yf.Ticker(ticker)
    hist = t.history(period='max', interval='1d')
    hist['Date'] = hist.index.strftime('%Y-%m-%d')
    hist['Ticker'] = ticker
    hist['LastUpdated'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    hist = hist.reset_index(drop=True)
    hist.to_sql('History', if_exists='append', index=False, con=engine, chunksize=1000)
    
def incremental_load(ticker):
    
    t = yf.Ticker(ticker)
    # Check if ticker is currently in history table. If not perform a full load
    if len(session.query(History).filter_by(Ticker="KINV-B.ST").all()) == 0:
        full_load(ticker)
        
    # I ticker is found in history table load from day after max Date.
    # Currently this shit has a bug loading the previous day... Due to weekend?!!!
    else:
        (Date, ) =session.query(func.max(History.Date)).filter_by(Ticker=ticker).one() # Get max Date for ticker
        start_date = (Date + timedelta(days=1)).strftime('%Y-%m-%d') # Set start date as day after max Date
        try:
            hist = t.history(start=start_date)
            hist['Date'] = hist.index.strftime('%Y-%m-%d')
            hist['Ticker'] = ticker
            hist['LastUpdated'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            hist = hist.reset_index(drop=True)
            hist.to_sql('History', if_exists='append', index=False, con=engine, chunksize=1000)
        except:
            pass

In [4]:
# Generate ticker list to be loaded for each stock index.
omxs30_tickers = pd.read_sql("SELECT Ticker FROM Companies WHERE List = 'OMXS30'", con=engine)['Ticker'].to_list()

In [5]:
Session = sessionmaker(bind=engine)
session = Session()

In [26]:
for t in omxs30_tickers:
    incremental_load(t)

- KINV-B.ST: Invalid input - start date cannot be after end date. startDate = 1669417200, endDate = 1669390223
- SWMA.ST: Invalid input - start date cannot be after end date. startDate = 1669417200, endDate = 1669390223
- SWED-A.ST: Invalid input - start date cannot be after end date. startDate = 1669417200, endDate = 1669390224
- TELIA.ST: Invalid input - start date cannot be after end date. startDate = 1669417200, endDate = 1669390224
- SCA-B.ST: Invalid input - start date cannot be after end date. startDate = 1669417200, endDate = 1669390224
- EVO.ST: Invalid input - start date cannot be after end date. startDate = 1669417200, endDate = 1669390224
- ELUX-B.ST: Invalid input - start date cannot be after end date. startDate = 1669417200, endDate = 1669390225
- SEB-A.ST: Invalid input - start date cannot be after end date. startDate = 1669417200, endDate = 1669390225
- GETI-B.ST: Invalid input - start date cannot be after end date. startDate = 1669417200, endDate = 1669390225
- NDA-SE.